In [2]:
from google.colab import drive

drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 33.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=945288674baa39cceb5c11e4d81a9f867855c342db0b070f644fefee11f9076b
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [1]:
import findspark
findspark.init()
import pyspark

from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession


sp = SparkContext()
ss = SparkSession(sp).builder.config("spark.driver.memory", "6g").getOrCreate()

import pyspark.sql.functions as f
#import pyspark

C:\Spark\spark-3.2.0-bin-hadoop3.2\python\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


http://localhost:4040/jobs/

In [2]:
#df = ss.read.csv("/gdrive/My Drive/BigData/2AMD15Milestone1Data").limit(10000)
df = ss.read.csv("stocks-data/MS1.txt")

In [3]:
df.show()

+--------------------+----------+-----+-----+
|                 _c0|       _c1|  _c2|  _c3|
+--------------------+----------+-----+-----+
|32843.Nordamerika...|01/01/2016|18.98|50635|
|32843.Nordamerika...|01/04/2016|18.52|51616|
|32843.Nordamerika...|01/05/2016|19.15|54898|
|32843.Nordamerika...|01/06/2016|19.71|41555|
|32843.Nordamerika...|01/07/2016|19.17|44430|
|32843.Nordamerika...|01/08/2016|18.94|72673|
|32843.Nordamerika...|01/11/2016| 19.1|45426|
|32843.Nordamerika...|01/12/2016|19.39|61457|
|32843.Nordamerika...|01/13/2016|19.27|61805|
|32843.Nordamerika...|01/14/2016|19.17|35597|
|32843.Nordamerika...|01/15/2016|18.81|69227|
|32843.Nordamerika...|01/18/2016|18.81|69227|
|32843.Nordamerika...|01/19/2016|18.82|23700|
|32843.Nordamerika...|01/20/2016|17.97|41439|
|32843.Nordamerika...|01/21/2016|17.82|35240|
|32843.Nordamerika...|01/22/2016|18.62|79139|
|32843.Nordamerika...|01/25/2016|18.09|89251|
|32843.Nordamerika...|01/26/2016|17.62|75300|
|32843.Nordamerika...|01/27/2016|1

The next function renames columns, format the date column and filter out the dates. Besides that, we drop the duplicates: records for the combination (stock, date) if there are more than one record per (stock, date).

In [4]:
from pyspark.sql.functions import date_format

newdf = df.select(df._c0.alias('stock_name'),
                  date_format(f.to_timestamp('_c1', 'mm/dd/yyyy'), "yyyy-mm-dd").alias('date'),
                  df._c2.alias('price'),
                  df._c3.alias('volume')).filter((f.year('date') >= 2016) & (f.year('date')<=2020))\
          .dropDuplicates(['stock_name', 'date'])

In [5]:
#newdf.count()

Get dates that have a lot of records 

In [5]:
dates = [row.date for row in newdf.groupby('date').count().where('count >= 25000').collect()]
dates.sort()
#len(dates)

Get the first 1005 dates 

In [6]:
dates2 = dates[:1005]

Collect the stocks that have enough data in these period

In [7]:
stocks_aligned = [row.stock_name for row in newdf.where(newdf.date.isin(dates2)).groupBy('stock_name').count().where('count >= 1000').collect()]

In [9]:
len(stocks_aligned)

1797

Select only limited number of stocks

In [8]:
chosen_stocks = stocks_aligned[:1000]
#len(chosen_stocks)

----

Basically, cell below was a trial to save corrected csv files

    from pyspark.sql.types import ArrayType, StringType

    def func():
        return dates2
    func_udf = f.udf(func, ArrayType(StringType()))


    for i, stock in enumerate(chosen_stocks):
        df_small = newdf.filter((newdf.stock_name == stock) & (newdf.date.isin(dates2)))
        df_base = df_small.select('stock_name').withColumn('date', f.explode(func_udf()))
        df_full = df_base.join(df_small, ['stock_name', 'date'], "leftouter")
    
    df_small.write.mode('overwrite').csv('backup_stocks/{}.csv'.format(i), header=True)
    # or do other union operation   

----

Create new dataframe that contains only chosen stocks within chosen dates

In [141]:
df_small = newdf.filter((newdf.stock_name.isin(chosen_stocks)) & (newdf.date.isin(dates2)))
#df_small.count()
df_small = df_small.repartition(2000)

In [142]:
df_small= df_small.withColumn('date', f.col('date'))\
                  .withColumn("date_existent", f.col("date"))

In [9]:
#df_small.write.mode('overwrite').csv('backup_stocks.csv', header=True)

Py4JJavaError: An error occurred while calling o3097.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Spark\spark-3.2.0-bin-hadoop3.2\python\lib\py4j-0.10.9.2-src.zip\py4j\clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "c:\program files\python36\lib\socket.py", line 586, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Spark\spark-3.2.0-bin-hadoop3.2\python\lib\py4j-0.10.9.2-src.zip\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\Spark\spark-3.2.0-bin-hadoop3.2\python\lib\py4j-0.10.9.2-src.zip\py4j\clientserver.py", line 504, in send_command
    "Error while sending or receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while sending or receivi

----

**Delete this part**

In [143]:
df_small = df_small.filter((df_small.stock_name.isin(chosen_stocks[:10])))

----

Now, we need to time-align it using User Defined Function of Spark. 

In [144]:
from pyspark.sql.types import ArrayType, StringType

In [145]:
def func():
    
    return dates2

In [146]:
func_udf = f.udf(func, ArrayType(StringType()))

The next cells basically creates the records with null values of price and value for combination of (stock, date) that were not in the original dataset

In [147]:
#full_df = ss.createDataFrame([], newdf.schema)
df_base = df_small.select('stock_name').distinct().withColumn('date', f.explode(func_udf()))\
                                                  .withColumn('date', f.col('date'))

In [148]:
df_full = df_base.join(df_small, ['stock_name', 'date'], "leftouter")

In [150]:
df_full.take(10)

Py4JJavaError: An error occurred while calling o14306.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 351.0 failed 1 times, most recent failure: Lost task 8.0 in stage 351.0 (TID 33063) (Kudakhter executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(Unknown Source)
	at java.nio.ByteBuffer.allocate(Unknown Source)
	at org.apache.spark.io.ReadAheadInputStream.<init>(ReadAheadInputStream.java:105)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:74)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:159)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.getSortedIterator(UnsafeExternalSorter.java:523)
	at org.apache.spark.sql.execution.UnsafeExternalRowSorter.sort(UnsafeExternalRowSorter.java:172)
	at org.apache.spark.sql.catalyst.expressions.Gen

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 49642)
Traceback (most recent call last):
  File "c:\program files\python36\lib\socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\program files\python36\lib\socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "c:\program files\python36\lib\socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\program files\python36\lib\socketserver.py", line 724, in __init__
    self.handle()
  File "C:\Spark\spark-3.2.0-bin-hadoop3.2\python\pyspark\accumulators.py", line 262, in handle
    poll(accum_updates)
  File "C:\Spark\spark-3.2.0-bin-hadoop3.2\python\pyspark\accumulators.py", line 235, in poll
    if func():
  File "C:\Spark\spark-3.2.0-bin-hadoop3.2\python\pyspark\accumulators.py", line 239, in accum_updates
 

In [94]:
df_full.count()

100500

In [48]:
df_full.columns

['stock_name', 'date', 'price', 'volume', 'date_existent']

In [95]:
df_full.filter(df_full.price.isNull()).count()

435

**TBD: Now, only fill the na values!**

Source: https://walkenho.github.io/interpolating-time-series-p2-spark/

In [133]:
from pyspark.sql import Window
import sys

In [134]:
window_ff = Window.partitionBy('stock_name')\
               .orderBy('date')\
               .rowsBetween(-sys.maxsize, 0)
               
window_bf = Window.partitionBy('stock_name')\
               .orderBy('date')\
               .rowsBetween(0, sys.maxsize)

In [135]:
read_last = f.last(df_full['price'], ignorenulls=True).over(window_ff)
readdate_last = f.last(df_full['date_existent'], ignorenulls=True).over(window_ff)

read_next = f.first(df_full['price'], ignorenulls=True).over(window_bf)
readdate_next = f.first(df_full['date_existent'], ignorenulls=True).over(window_bf)

df_filled = df_full.withColumn('price_ff', read_last)\
                        .withColumn('date_ff', readdate_last)\
                        .withColumn('price_bf', read_next)\
                        .withColumn('date_bf', readdate_next)

In [136]:
from pyspark.sql.types import FloatType

In [137]:
import datetime

def interpol(x, x_prev, x_next, y_prev, y_next, y):
    if x_prev == x_next:
        return y
    else:
        m = (float(y_next)-float(y_prev))/(datetime.date(int(x_next[:3]), int(x_next[5:7]), int(x_next[-2:])) - datetime.date(int(x_prev[:3]), int(x_prev[5:7]), int(x_prev[-2:]))).days
        y_interpol = float(y_prev) + m * (datetime.date(int(x[:3]), int(x[5:7]), int(x[-2:])) - datetime.date(int(x_prev[:3]), int(x_prev[5:7]), int(x_prev[-2:]))).days
        return y_interpol

interpol_udf = f.udf(interpol, FloatType())

In [138]:
df_filled = df_filled.withColumn('price_interpol',
                                 interpol_udf('date', 'date_ff', 'date_bf', 'price_ff', 'price_bf', 'price'))\
                     .drop('date_existent', 'date_ff', 'date_bf')\
                     .withColumn('date', f.from_unixtime(f.col('date')))

In [74]:
df_filled.columns

['stock_name',
 'date',
 'price',
 'volume',
 'price_ff',
 'price_bf',
 'price_interpol']

In [139]:
df_filled.filter(df_filled.price_interpol.isNull()).count()

100065

In [140]:
df_filled.take(5)

[Row(stock_name='10081.Asien--Australien--Südamerika--Afrika_Japan_Riso-Kyoiku_0490ZB', date=None, price='348', volume='126400', price_ff='348', price_bf='348', price_interpol=None),
 Row(stock_name='10081.Asien--Australien--Südamerika--Afrika_Japan_Riso-Kyoiku_0490ZB', date=None, price='351', volume='107400', price_ff='351', price_bf='351', price_interpol=None),
 Row(stock_name='10081.Asien--Australien--Südamerika--Afrika_Japan_Riso-Kyoiku_0490ZB', date=None, price='347', volume='112800', price_ff='347', price_bf='347', price_interpol=None),
 Row(stock_name='10081.Asien--Australien--Südamerika--Afrika_Japan_Riso-Kyoiku_0490ZB', date=None, price='346', volume='194400', price_ff='346', price_bf='346', price_interpol=None),
 Row(stock_name='10081.Asien--Australien--Südamerika--Afrika_Japan_Riso-Kyoiku_0490ZB', date=None, price='337', volume='176900', price_ff='337', price_bf='337', price_interpol=None)]

#### Working with weather dataset

In [59]:
#df2 = ss.read.csv("/gdrive/My Drive/BigData/weather/*", header=False).limit(1000)
files_list = ['weather-data/out_2016.txt', 
              'weather-data/out_2017.txt',
              'weather-data/out_2018.txt',
              'weather-data/out_2019.txt',
              'weather-data/out_2020.txt']
df2 = ss.read.csv(files_list, header=False)

In [60]:
df2.count()

22117540

In [61]:
df2 = df2.filter(df2._c0 != 'STATION_ID')

Next, parse the columns to get all the quantities into a separate columns

In [62]:
split_col = f.split(df2['_c3'], ' ')

df2 = df2.withColumn('wind_angle', split_col.getItem(0))
df2 = df2.withColumn('wind_qa', split_col.getItem(1))
df2 = df2.withColumn('wind_type', split_col.getItem(2))
df2 = df2.withColumn('wind_speed', split_col.getItem(3))
df2 = df2.withColumn('wind_qs', split_col.getItem(4))

In [63]:
split_col = f.split(df2['_c4'], ' ')

df2 = df2.withColumn('tmp_val', split_col.getItem(0))
df2 = df2.withColumn('tmp_q', split_col.getItem(1))

In [64]:
split_col = f.split(df2['_c5'], ' ')

df2 = df2.withColumn('dew_val', split_col.getItem(0))
df2 = df2.withColumn('dew_q', split_col.getItem(1))

In [65]:
split_col = f.split(df2['_c6'], ' ')

df2 = df2.withColumn('slp_val', split_col.getItem(0))
df2 = df2.withColumn('slp_q', split_col.getItem(1))

In [66]:
df2 = df2.drop('_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13')

In [12]:
df2.show()

+-----------+----------+--------------------+----------+-------+---------+----------+-------+-------+-----+-------+-----+-------+-----+
|        _c0|       _c1|                 _c2|wind_angle|wind_qa|wind_type|wind_speed|wind_qs|tmp_val|tmp_q|dew_val|dew_q|slp_val|slp_q|
+-----------+----------+--------------------+----------+-------+---------+----------+-------+-------+-----+-------+-----+-------+-----+
|06022499999|2016-01-01|HARALD HELIDECK O...|       210|      1|        N|      0113|      1|  +0070|    1|  +0060|    1|  99999|    9|
|06022499999|2016-01-02|HARALD HELIDECK O...|       130|      1|        N|      0118|      1|  +0070|    1|  +0030|    1|  99999|    9|
|06022499999|2016-01-03|HARALD HELIDECK O...|       110|      1|        N|      0221|      1|  +0020|    1|  -0010|    1|  99999|    9|
|06022499999|2016-01-04|HARALD HELIDECK O...|       110|      1|        N|      0211|      1|  +0020|    1|  +0010|    1|  99999|    9|
|06022499999|2016-01-05|HARALD HELIDECK O...|   

## TO DO:

- parse the columns of temperature and other values to float

rename the column to date

In [67]:
df2 = df2.select('*', df2._c1.alias('date'))

Filter the dataset according to the date range we have

In [68]:
df2 = df2.filter(df2.date.isin(dates2))

Drop duplicates

In [72]:
df3 = df2.dropDuplicates(['_c0', 'date']).groupby('_c0').count()

In [76]:
station_ids = [row._c0 for row in df3.filter('count >= 1000').collect()]

----

As we do not have the data for all the stations for the chosen period of dates, we need to fill the missing values as well.

First of all, let's time align the weather dataset in the same way we've done with stocks

In [77]:
weather_small = df2.filter(df2._c0.isin(station_ids[:1000]))

In [78]:
weather_small= weather_small.withColumn('date', f.col('date'))\
                            .withColumn("date_existent", f.col("date"))

In [79]:
w_base = weather_small.select('_c0').distinct().withColumn('date', f.explode(func_udf()))\
                                               .withColumn('date', f.col('date'))

In [80]:
weather_full = w_base.join(weather_small, ['_c0', 'date'], "leftouter")

In [81]:
weather_full.count()

1005000

In [84]:
weather_full.filter(weather_full.tmp_val.isNull()).count()

1005000

-----

In [16]:
newdf = newdf.join(df2, on=['date'], how='inner')